In [1]:
import sys
sys.path.append("..")

from src import bioinfo_tools as bt
from src import bioinfo_sequence_functions as bsf

import pandas as pd

In [2]:
pdf_proteins = pd.read_csv("../data/proteins_86693_757732.csv")
dict_sequence = bt.read_fasta_2dictionary("../data/sequence.fasta")

# B. Get Statistics
The genomic sequence should be read only in the positive sense, i.e. from 5’ to 3’. Read the
genomic sequence and obtain the following statistics:
1. Length of the sequence.
2. Frequency (in %) of A, C, G, T.
3. GC content.
4. Number of Start (AUG) codons found.
5. Number of Stop Codons (UAA, UAG, UGA).
6. Most and less frequent codon.

In [3]:
sequence = dict_sequence["NC_045512.2"]

In [232]:
def number_of_proteins_by_frame(seq):
  lt_rf = bsf.reading_frames(seq)
  lt_prots = list(map(bsf.all_proteins_rf, lt_rf))

  print("-> Number of possible proteins:")
  print("  > direct sequence")
  j = 0
  for i in range(0, len(lt_prots)):
    print(f"    * In frame {j}: {len(lt_prots[i])}")
    j+=1
    if i == 2: 
      j=0
      print("  > reverse sequence")

In [233]:
number_of_proteins_by_frame(sequence)

-> Number of possible proteins:
  > direct sequence
    * In frame 0: 116
    * In frame 1: 311
    * In frame 2: 297
  > reverse sequence
    * In frame 0: 176
    * In frame 1: 136
    * In frame 2: 172


In [114]:
for i in range(0,3):
    l_start = len(bsf.start_codon_position(sequence, i))
    l_end = len(bsf.start_codon_position(bsf.reverse_complement(sequence), i))
    print(f"* In frame {i}: \n  - Start: {l_start} | Stop: {l_end}")

* In frame 0: 
  - Start: 117 | Stop: 176
* In frame 1: 
  - Start: 311 | Stop: 136
* In frame 2: 
  - Start: 297 | Stop: 172


In [7]:
[len(bsf.start_codon_position(bsf.reverse_complement(sequence), i)) for i in range(0,3)]

[176, 136, 172]

In [235]:
freq = bsf.frequency(sequence, 1, 1)
total = sum(list(freq.values()))

In [48]:
dict_freq_codons = bsf.frequency(sequence, 3, 3)
lt = list(dict_freq_codons.values())
lt_max = max(lt)
lt_min = min(lt)

[(i,j) for i, j in  dict_freq_codons.items() if j == lt_max or j == lt_min]

[('TTT', 383), ('GCG', 22)]

# C. Get ORFs
Identify all potential ORFs. Using the complete genome sequence as input, locate all the
potential ORFs in the positive sense.
- An ORF is defined as the region that starts with the start codon (AUG) and ends
with the stop codon (UAA, UAG, UGA).
- For a given region, if alternative start codons are found, select the longest ORF.
- Select all ORFs with a minimum length of 120 nucleotides (40 amino acids).

In this step, you should output the following information:

7. A file with all the protein sequences named all_potential_proteins.txt, with a
sequence per line.
8. A file with the genomic coordinates of all the ORFs, named orf_coordinates.txt. </br>
The genomic coordinates correspond the start and end position in the genome in the
format:  </br>
```
Start1, End1, ORF1
Start2, End2, ORF2
…..
StartN, EndN, ORFN


In [27]:
import importlib
importlib.reload(bsf)

<module 'src.bioinfo_sequence_functions' from '../src/bioinfo_sequence_functions.py'>

## Roots way

In [54]:
lt_proteins = bsf.all_orfs_ord(sequence, 40)

In [55]:
# filename = "all_potential_proteins"
# with open(f"../data/{filename}.txt", "w") as f:
#     f.write("\n".join(lt_proteins))   

In [56]:
lt_reading_frames = bsf.reading_frames(sequence)

lt_proteins_and_positions = list(map(lambda x: bsf.all_proteins_rf(x, True), lt_reading_frames))

lt_proteins_and_positions = [
    list(
        zip(
            lt_proteins_and_positions[i][0], 
            lt_proteins_and_positions[i][1]
        )
    ) 
    for i in range(0, len(lt_reading_frames))
]


In [57]:
num_aminoacids = 40
f = lambda x: (
    sorted(
        list(filter(lambda y: len(y[0]) >= num_aminoacids, x)), 
            key=lambda z: len(z[0]),
            reverse=True
    )
)

lt_positions_start_end = list(map(f, lt_proteins_and_positions))

In [77]:
lt_temp = sorted(lt_positions_start_end[0], key=lambda x: x[1])
bt.get_orf_names(list(zip(*lt_temp))[1])

In [423]:
# def organize_coordinates(x):
#     x_lag1 = x.copy()
#     x_lag1.pop(0)
#     x_lag1.append(("", [0,0]))

#     switch = False
#     k = 0
#     res = []
#     for i in range(0, len(x)):
#         if x[i][1][1] == x_lag1[i][1][1]:
#             if not switch:
#                 switch = True
#                 j = 0
#             else:
#                 j += 1
#             row = f"{x[i][1][0]*3}, {x[i][1][1]*3+4}, ORF{k}.{j}"
#         else:
#             switch = False
#             k += 1
#             row = f"{x[i][1][0]*3}, {x[i][1][1]*3+4}, ORF{k}"
#         res.append(row)
#     return res

In [424]:
# f = lambda x: [f"{x[i][1][0]*3}, {x[i][1][1]*3+4}, ORF{i}" for i in range(0, len(x))]
# lt_positions_to_txt = list(map(organize_coordinates, lt_positions_start_end))

In [81]:
# lt_positions_to_txt[1]

In [331]:
j = 0
for i in range(0, len(lt_positions_to_txt)):
    if i <= 2:
        suffix = f"direct_frame_0{i}"
    else:
        suffix = f"reverse_frame_0{j}"
        j += 1
    filename = f"orf_coordinates_{suffix}"
    with open(f"../data/{filename}.txt", "w") as f:
        f.write("\n".join(lt_positions_to_txt[i]))

## Pandas way

In [196]:
import pandas as pd

In [346]:
import importlib
importlib.reload(bsf)

<module 'src.bioinfo_sequence_functions' from '../src/bioinfo_sequence_functions.py'>

In [459]:
''' 
=====================================//=====================================
STEP 01 - Creating main dataset
=====================================//=====================================
In this step, we are creating a dataframe that consolidates all the information 
regarding the ORFs and their respective positions, for each of the possible DNA 
sequences (3 direct frames and 3 reverse frames).
------------------------------------//--------------------------------------
'''
#  Getting 6 sequences, one for each direct and reverse reading frame:
lt_reading_frames = bsf.reading_frames(sequence)
# Putting those sequences into a Pandas DataFrame with "reading_frame" as index:
pdf = pd.DataFrame(lt_reading_frames, columns=["seq__amino"])
pdf.index.names = ["reading_frame"]
# Getting all ORFs associated with those 6 sequences with "all_proteins_rf" function;
# Creating two new columns with the outputs of "all_proteins_rf" (orfs and positions);
# Getting only the ORFs that have the longest length when ORF overlap occurs (max_len):
pdf["seq__orf"], pdf["orf__coord"] = zip(*pdf.apply(lambda x: bsf.all_proteins_rf(x["seq__amino"], True, max_len=False), axis=1))
# Exploding the proteins and positions calculated above into rows:
pdf_exp = pdf.explode(["seq__orf", "orf__coord"])
pdf_exp["orf__coord_start"], pdf_exp["orf__coord_end"] = zip(*pdf_exp["orf__coord"])
# Creating a column with the length of each ORF:
pdf_exp["seq__orf_len"] = pdf_exp["seq__orf"].str.len()
# Creating two new columns for dna coordinates calculated from orf/aminoacid coordinates:
for n in [0,1,2]:
    pdf_exp.loc[n, "dna__coord_start"] = pdf_exp.loc[n, "orf__coord_start"]*3 + n
    pdf_exp.loc[n, "dna__coord_end"] = pdf_exp.loc[n, "orf__coord_end"]*3 + 3 + n
#droping reading frames of the reverse sequence
pdf_exp = pdf_exp.dropna()
# Getting the DNA sub-sequences associated with start and end of each ORF:
pdf_exp["seq__dna"] = pdf_exp.apply(lambda x: sequence[int(x["dna__coord_start"]):int(x["dna__coord_end"])], axis=1)

In [460]:
'''
=====================================//=====================================
STEP 02 - Applying filters
=====================================//=====================================
In this step, we are applying all the filters stipulated by the proposed 
exercise, namely:
(1) sequences must contain >= 40 amino acids;
(2) If an ORF is contained within another, select only the largest of them;
(3) order the data by ORF length.
------------------------------------//--------------------------------------
'''
# Creating a new dataframe "prf_exp_temp" to apply filters and other particular manipulations:
pdf_exp_temp = pdf_exp.copy()
# Filtering only ORFs with at least 40 aminoacis in its sequence:
pdf_exp_temp = pdf_exp_temp[pdf_exp_temp["seq__orf"].str.len() >= 40]
# Ordering dataframe by ORF coordinates to apply the function that will create "names"/"labes" for those ORFs:
names_by_frame = True
if names_by_frame:
    pdf_exp_temp = pdf_exp_temp.reset_index().sort_values(["reading_frame","orf__coord_start","orf__coord_end"], ascending=[True,True,False]).set_index("reading_frame")
    lt_orf_names = pdf_exp_temp.groupby("reading_frame").apply(lambda x: bt.get_orf_names(x[["dna__coord_start", "dna__coord_end"]].values))
    lt_orf_names = [[f"rf0{j}_" + i for i in lt_orf_names[j]] for j in range(0, len(lt_orf_names))]
    pdf_exp_temp["orf__name"] = sum(lt_orf_names, [])
else:
    pdf_exp_temp = pdf_exp_temp.sort_values("orf__coord")
    pdf_exp_temp["orf__name"] = bt.get_orf_names(pdf_exp_temp[["dna__coord_start", "dna__coord_end"]].values)
# Sorting dataframe by orf length:
pdf_exp_temp = pdf_exp_temp.sort_values("seq__orf_len", ascending=False)
# Just sorting columns alphabeticaly:
pdf_exp_temp = pdf_exp_temp[sorted(pdf_exp_temp.columns)]

## 7.
A file with all the protein sequences named all_potential_proteins.txt, with a
sequence per line.

In [494]:
(
    pdf_exp_temp
        .sort_values("seq__orf_len", ascending=False)["seq__dna"]
        .to_csv("../data/all_potential_proteins.txt", header=False, index=False)
)

## 8. 
A file with the genomic coordinates of all the ORFs, named orf_coordinates.txt. </br>
The genomic coordinates correspond the start and end position in the genome in the
format:  </br>
```
Start1, End1, ORF1
Start2, End2, ORF2
…..
StartN, EndN, ORFN


In [495]:
import re
# pattern = "ORF\d+\.[a-z]|ORF\d+"
# pattern = "ORF\d+"
pattern = "rf0\d{1}_ORF\d+"
l = pd.Series([re.findall(pattern, orf)[0] for orf in pdf_exp_temp["orf__name"] if re.findall(pattern, orf)]).unique()
print(len(pdf_exp_temp[pdf_exp_temp["orf__name"].isin(l)].sort_values("orf__coord")))
pdf_exp_temp_ex08 = pdf_exp_temp[pdf_exp_temp["orf__name"].isin(l)].sort_values("orf__coord")

34


In [496]:
pdf_exp_temp_ex08 = pdf_exp_temp_ex08.sort_values("seq__orf_len", ascending=False)

In [497]:
(
    pdf_exp_temp_ex08[["dna__coord_start", "dna__coord_end", "seq__dna"]]
        .to_csv(
            "../data/orf_coordinates.txt", 
            float_format="%.0f", 
            sep=",", 
            index=False, 
            header=False
        )
)

In [ ]:
import re
# pattern = "ORF\d+\.[a-z]|ORF\d+"
# pattern = "ORF\d+"
pattern = "rf0\d{1}_ORF\d+"
l = pd.Series([re.findall(pattern, orf)[0] for orf in pdf_exp_temp["orf__name"] if re.findall(pattern, orf)]).unique()
print(len(pdf_exp_temp[pdf_exp_temp["orf__name"].isin(l)].sort_values("orf__coord")))
pdf_exp_temp_ex08 = pdf_exp_temp[pdf_exp_temp["orf__name"].isin(l)].sort_values("orf__coord")
pdf_exp_temp_ex08 = pdf_exp_temp_ex08.sort_values("seq__orf_len", ascending=False)
(
    pdf_exp_temp_ex08[["dna__coord_start", "dna__coord_end", "seq__dna"]]
        .to_csv(
            "../data/orf_coordinates.txt", 
            float_format="%.0f", 
            sep=",", 
            index=False, 
            header=False
        )
)

## Temp

In [178]:
def all_proteins_rf(aa_seq, return_positions=False):
    """Computes all posible proteins in an aminoacid sequence."""
    aa_seq = aa_seq.upper()
    current_prot = []
    proteins = []
    positions = []
    current_pos = []
    for aa in range(0, len(aa_seq)):
        if aa_seq[aa] == "_":
            if current_prot:
                current_pos.append(aa)
                for i in range(0, len(current_prot)):
                    proteins.append(current_prot[i])
                    positions.append([current_pos[i], current_pos[-1]])
                current_prot = []
                current_pos = []
        else:
            if aa_seq[aa] == "M":
                current_prot.append("")
                current_pos.append(aa)
            for i in range(len(current_prot)):
                current_prot[i] += aa_seq[aa]
                # current_pos.append(aa)
    # proteins = list(filter(lambda x: len(x)>=2, proteins))
    # positions = list(filter(lambda x: len(x)>=2, positions))
    if return_positions:
        return proteins, positions
    else: 
        return proteins

In [182]:
len(all_proteins_rf(lt_reading_frames[0], return_positions=False))

116

In [217]:
def finding_ORFs(dna_seq, pos_ini = 0, min_size=120):
    actual_orf = []
    orfs = {}
    new_orfs={}
    start_protein=[]
    for i in range(pos_ini,(len(dna_seq)-3 + 1),3):
        codon = dna_seq[i:i+3]
        if codon=="UAA" or codon=="UAG" or codon=="UGA":
            if actual_orf:
                sorted(actual_orf, key=lambda x: len(x), reverse=True)
                start_protein_sorted=sorted(start_protein)
                end_protein=(i+3)
                new_orfs[actual_orf[0]]=[start_protein_sorted[0],end_protein]
                for key in new_orfs:
                    if len(key) >= min_size:
                        orfs[key] = new_orfs[key]
                actual_orf = [] 
                start_protein=[]
        else:
            if codon == "AUG":
                start_protein.append(i+1)
                actual_orf.append("")
            for i in range(len(actual_orf)):
                actual_orf[i] += codon
    return orfs

In [218]:
res = []
for i in range(0, 3):
    res.append(finding_ORFs(sequence.replace("T","U"), pos_ini = i, min_size=120))

In [219]:
len(res[0])

6

In [220]:
len(res[1])

8

In [221]:
len([2])

1

# D. Overlap with annotation
Compare the results you obtained with those from the annotation in file
proteins_86693_757732.csv (see Table 1). This file contains the genomic coordinates of
the ORFs that code for the different proteins. For instance, the ORFs that code for the
Spike Protein (S) is (start=)21563 (end=)25384. This represents the start and end of the
genomic coordinates of this ORF. Check the overlap of your ORFs with those in this
table.

The overlap between sequences A and B can be calculated as:
$$overlap(A,B) = \frac{|A \cap B|}{min(A,B)}$$

where the intersection region between A and B is defined by
the length of the genomic region in common between A and B. For this exercise, we will
define a slightly different version of the overlap, where A is one of the annotated ORF in
Table 1 and B is an ORF from your list. Thus, the overlap is defined taking in account the
length of the ORF in Table 1.
$$overlap(A,B) = \frac{|A \cap B|}{|A|}$$

In [498]:
pdf_proteins["seq__dna"] = pdf_proteins.apply(lambda x: sequence[int(x["Start"])-1:int(x["Stop"])], axis=1)

In [499]:
pdf_proteins["dna__coord_start"] = pdf_proteins["Start"] - 1
pdf_proteins["dna__coord_end"] = pdf_proteins["Stop"]

In [ ]:
pdf_proteins["seq__dna"] = pdf_proteins.apply(lambda x: sequence[int(x["Start"])-1:int(x["Stop"])], axis=1)
pdf_proteins["dna__coord_start"] = pdf_proteins["Start"] - 1
pdf_proteins["dna__coord_end"] = pdf_proteins["Stop"]
pdf_temp = (
    pdf_proteins[
        ["Locus", "dna__coord_start", "dna__coord_end", "seq__dna"]
        ]
    .merge(
        pdf_exp_temp[
            ["seq__dna", "dna__coord_start", "dna__coord_end", "orf__name"]
            ], 
        how="cross"
        )
    )

import difflib

def get_overlap(s1, s2):
    s = difflib.SequenceMatcher(None, s1, s2)
    pos_a, pos_b, size = s.find_longest_match(0, len(s1), 0, len(s2)) 
    return s1[pos_a:pos_a+size]

pdf_temp["overlap__seq"] = pdf_temp.apply(lambda x: get_overlap(str(x["seq__dna_x"]), str(x["seq__dna_y"])), axis=1)

pdf_temp["overlap__perc"] = pdf_temp["overlap__seq"].str.len()/pdf_temp["seq__dna_x"].str.len()

In [500]:
# df_joined = pdf_proteins.merge(pdf_exp_temp[[]], on=["dna__coord_start", "dna__coord_end"], how="left")

In [516]:
pdf_temp = (
    pdf_proteins[
        ["Locus", "dna__coord_start", "dna__coord_end", "seq__dna"]
        ]
    .merge(
        pdf_exp_temp[
            ["seq__dna", "dna__coord_start", "dna__coord_end", "orf__name"]
            ], 
        how="cross"
        )
    )

In [517]:
# (
#     pdf_proteins[["Locus tag", "dna__coord_start", "dna__coord_end", "seq__dna"]]
#         .merge(
#             pdf_exp_temp[["seq__dna", "dna__coord_start", "dna__coord_end", "orf__name"]], 
#             on = ["dna__coord_start", "dna__coord_end"],
#             how="left")
#             )

In [532]:
# https://stackoverflow.com/questions/14128763/how-to-find-the-overlap-between-2-sequences-and-return-it
# def longest_common_substring(S1, S2):
#   M = [[0]*(1+len(S2)) for i in range(1+len(S1))]
#   longest, x_longest = 0, 0
#   for x in range(1,1+len(S1)):
#     for y in range(1,1+len(S2)):
#         if S1[x-1] == S2[y-1]:
#             M[x][y] = M[x-1][y-1] + 1
#             if M[x][y]>longest:
#                 longest = M[x][y]
#                 x_longest  = x
#         else:
#             M[x][y] = 0
#   return S1[x_longest-longest: x_longest]

import difflib

def get_overlap(s1, s2):
    s = difflib.SequenceMatcher(None, s1, s2)
    pos_a, pos_b, size = s.find_longest_match(0, len(s1), 0, len(s2)) 
    return s1[pos_a:pos_a+size]

In [519]:
pdf_temp["overlap__seq"] = pdf_temp.apply(lambda x: get_overlap(str(x["seq__dna_x"]), str(x["seq__dna_y"])), axis=1)

In [520]:
pdf_temp["overlap__perc"] = pdf_temp["overlap__seq"].str.len()/pdf_temp["seq__dna_x"].str.len()

## 9.
For each of the ORFs in table 1 you should output the longest overlap obtained with
an ORF in your list. Use the identifiers in the column “Locus” to identify the ORF, e.g.
```
ORF1ab 72%
S      93%
. . . 
```
So, in the case above ORF1ab has an overlap of 72% with one of your ORFs, being 72
the percentage corresponding to the longest overlap. Hint: Define a function that
compares the overlap between two sequences as defined above and test for each
ORF in Table 1 the overlap with each of the ORFs in your list.

In [ ]:
pdf_temp.sort_values(["Locus", "overlap__perc"], ascending=[True, False]).groupby("Locus").head(1)[["Locus","overlap__perc"]]

In [531]:
pdf_temp.sort_values(["Locus", "overlap__perc"], ascending=[True, False]).groupby("Locus").head(1)[["Locus","overlap__seq","orf__name","overlap__perc"]]

,Locus,overlap__seq,orf__name,overlap__perc
1151,E,ATGTACTCATTCGTTTCGGAAGAGACAGGTACGTTAATAGTTAATA...,rf00_ORF4,1.000000
1368,M,ATGGCAGATTCCAACGGTACTATTACCGTTGAAGAGCTTAAAAAGC...,rf02_ORF16,1.000000
2536,N,ATGTCTGATAATGGACCCCAAAATCAGCGAAATGCACCCCGCATTA...,rf01_ORF8,1.000000
2833,ORF10,TAACTTTA,rf01_ORF7,0.068376
239,ORF1ab,ATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAAC...,rf01_ORF1,1.000000
882,ORF3a,ATGGATTTGTTTATGAGAATCTTCACAATTGGAACTGTAACTTTGA...,rf00_ORF3,1.000000
1636,ORF6,ATGTTTCATCTCGTTGACTTTCAGGTTACTATAGCAGAGATATTAC...,rf00_ORF5,1.000000
1855,ORF7a,ATGAAAATTATTCTTTTCTTGGCACTGATAACACTCGCTACTTGTG...,rf00_ORF6,1.000000
2139,ORF7b,ATGATTGAACTTTCATTAATTGACTTCTATTTGTGCTTTTTAGCCT...,rf02_ORF17,1.000000
2332,ORF8,ATGAAATTTCTTGTTTTCTTAGGAATCATCACAACTGTAGCTGCAT...,rf02_ORF18,1.000000


## 10. Output
For this exercise, you should submit a file name a python script named **run.py**. Note that all
submissions in different format will not be considered! The file should be run as:
```
Python run.py sequence.fasta
```
Points 1 to 6 and 9, should be written to output, with the one item per line. </br>
Points 7 and 8 should save the results in files with the names as indicated above.

In [28]:
def main(sequence):
    '''
    =====================================//=====================================
    STEP 00 - Get statistics
    =====================================//=====================================
    In this step, we are applying all the filters stipulated by the proposed 
    exercise, namely:
    ------------------------------------//--------------------------------------
    '''
    # 1) Length of the sequence.
    print(f"\n -> Sequence len: {len(sequence)}")

    # 2) Frequency in %:
    print(f"\n -> Frequence:")
    freq = bsf.frequency(sequence, 1, 1)
    total = sum(list(freq.values()))
    for i, j in freq.items():
        print(f"    {i}: {str(round(j/total*100, 2))}%")

    # 3) GC content:
    print(f"\n -> GC content: {round(bsf.gc_content(sequence)*100, 2)}%")

    # 4,5) Number of start/end codon:
    print("\n -> Number of start/end codons:")
    for i in range(0,3):
        l_start = len(bsf.start_codon_position(sequence, i))
        l_end = len(bsf.start_codon_position(bsf.reverse_complement(sequence), i))
        print(f"    * In frame {i}: \n      - Start: {l_start} | Stop: {l_end}")

    # 6) Most and less freq. codon:
    dict_freq_codons = bsf.frequency(sequence, 3, 1)
    lt = list(dict_freq_codons.values())
    lt_max = max(lt)
    lt_min = min(lt)

    print("\n -> Most and less freq. codon:")
    for i, j in  dict_freq_codons.items():
        if j == lt_max or j == lt_min:
            print(f"    {i}: {j}")
    
    ''' 
    =====================================//=====================================
    STEP 01 - Creating main dataset
    =====================================//=====================================
    In this step, we are creating a dataframe that consolidates all the information 
    regarding the ORFs and their respective positions, for each of the possible DNA 
    sequences (3 direct frames and 3 reverse frames).
    ------------------------------------//--------------------------------------
    '''
    #  Getting 6 sequences, one for each direct and reverse reading frame:
    lt_reading_frames = bsf.reading_frames(sequence)
    # Putting those sequences into a Pandas DataFrame with "reading_frame" as index:
    pdf = pd.DataFrame(lt_reading_frames, columns=["seq__amino"])
    pdf.index.names = ["reading_frame"]
    # Getting all ORFs associated with those 6 sequences with "all_proteins_rf" function;
    # Creating two new columns with the outputs of "all_proteins_rf" (orfs and positions);
    # Getting only the ORFs that have the longest length when ORF overlap occurs (max_len):
    pdf["seq__orf"], pdf["orf__coord"] = zip(*pdf.apply(lambda x: bsf.all_proteins_rf(x["seq__amino"], True, max_len=False), axis=1))
    # Exploding the proteins and positions calculated above into rows:
    pdf_exp = pdf.explode(["seq__orf", "orf__coord"])
    pdf_exp["orf__coord_start"], pdf_exp["orf__coord_end"] = zip(*pdf_exp["orf__coord"])
    # Creating a column with the length of each ORF:
    pdf_exp["seq__orf_len"] = pdf_exp["seq__orf"].str.len()
    # Creating two new columns for dna coordinates calculated from orf/aminoacid coordinates:
    for n in [0,1,2]:
        pdf_exp.loc[n, "dna__coord_start"] = pdf_exp.loc[n, "orf__coord_start"]*3 + n
        pdf_exp.loc[n, "dna__coord_end"] = pdf_exp.loc[n, "orf__coord_end"]*3 + 3 + n
    #droping reading frames of the reverse sequence
    pdf_exp = pdf_exp.dropna()
    # Getting the DNA sub-sequences associated with start and end of each ORF:
    pdf_exp["seq__dna"] = pdf_exp.apply(lambda x: sequence[int(x["dna__coord_start"]):int(x["dna__coord_end"])], axis=1)

    '''
    =====================================//=====================================
    STEP 02 - Applying filters
    =====================================//=====================================
    In this step, we are applying all the filters stipulated by the proposed 
    exercise, namely:
    (1) sequences must contain >= 40 amino acids;
    (2) If an ORF is contained within another, select only the largest of them;
    (3) order the data by ORF length.
    ------------------------------------//--------------------------------------
    '''
    # Creating a new dataframe "prf_exp_temp" to apply filters and other particular manipulations:
    pdf_exp_temp = pdf_exp.copy()
    # Filtering only ORFs with at least 40 aminoacis in its sequence:
    pdf_exp_temp = pdf_exp_temp[pdf_exp_temp["seq__orf"].str.len() >= 40]
    # Ordering dataframe by ORF coordinates to apply the function that will create "names"/"labes" for those ORFs:
    names_by_frame = True
    if names_by_frame:
        pdf_exp_temp = pdf_exp_temp.reset_index().sort_values(["reading_frame","orf__coord_start","orf__coord_end"], ascending=[True,True,False]).set_index("reading_frame")
        lt_orf_names = pdf_exp_temp.groupby("reading_frame").apply(lambda x: bt.get_orf_names(x[["dna__coord_start", "dna__coord_end"]].values))
        lt_orf_names = [[f"rf0{j}_" + i for i in lt_orf_names[j]] for j in range(0, len(lt_orf_names))]
        pdf_exp_temp["orf__name"] = sum(lt_orf_names, [])
    else:
        pdf_exp_temp = pdf_exp_temp.sort_values("orf__coord")
        pdf_exp_temp["orf__name"] = bt.get_orf_names(pdf_exp_temp[["dna__coord_start", "dna__coord_end"]].values)
    # Sorting dataframe by orf length:
    pdf_exp_temp = pdf_exp_temp.sort_values("seq__orf_len", ascending=False)
    # Just sorting columns alphabeticaly:
    pdf_exp_temp = pdf_exp_temp[sorted(pdf_exp_temp.columns)]
    
    print("\n -> creating all_potential_proteins.txt file...")
    (
        pdf_exp_temp
            .sort_values("seq__orf_len", ascending=False)["seq__dna"]
            .to_csv("../data/all_potential_proteins.txt", header=False, index=False)
    )
    print("    * file created!")

    print("\n -> creating orf_coordinates.txt file...")
    import re
    pattern = "rf0\d{1}_ORF\d+"
    l = pd.Series([re.findall(pattern, orf)[0] for orf in pdf_exp_temp["orf__name"] if re.findall(pattern, orf)]).unique()
    pdf_exp_temp_ex08 = pdf_exp_temp[pdf_exp_temp["orf__name"].isin(l)].sort_values("orf__coord")
    pdf_exp_temp_ex08 = pdf_exp_temp_ex08.sort_values("seq__orf_len", ascending=False)
    (
        pdf_exp_temp_ex08[["dna__coord_start", "dna__coord_end", "seq__dna"]]
            .to_csv(
                "../data/orf_coordinates.txt", 
                float_format="%.0f", 
                sep=",", 
                index=False, 
                header=False
            )
    )
    print("    * file created!")

    '''
    =====================================//=====================================
    STEP 03 - Applying filters
    =====================================//=====================================
    In this step, we are applying all the filters stipulated by the proposed 
    exercise, namely:
    (1) sequences must contain >= 40 amino acids;
    (2) If an ORF is contained within another, select only the largest of them;
    (3) order the data by ORF length.
    ------------------------------------//--------------------------------------
    '''
    pdf_proteins["seq__dna"] = pdf_proteins.apply(lambda x: sequence[int(x["Start"])-1:int(x["Stop"])], axis=1)
    pdf_proteins["dna__coord_start"] = pdf_proteins["Start"] - 1
    pdf_proteins["dna__coord_end"] = pdf_proteins["Stop"]
    pdf_temp = (
        pdf_proteins[
            ["Locus", "dna__coord_start", "dna__coord_end", "seq__dna"]
            ]
        .merge(
            pdf_exp_temp[
                ["seq__dna", "dna__coord_start", "dna__coord_end", "orf__name"]
                ], 
            how="cross"
            )
        )

    import difflib

    def get_overlap(s1, s2):
        s = difflib.SequenceMatcher(None, s1, s2)
        pos_a, pos_b, size = s.find_longest_match(0, len(s1), 0, len(s2)) 
        return s1[pos_a:pos_a+size]

    print("\n -> Getting overlap between ORFs...")
    pdf_temp["overlap__seq"] = pdf_temp.apply(lambda x: get_overlap(str(x["seq__dna_x"]), str(x["seq__dna_y"])), axis=1)
    pdf_temp["overlap__perc"] = pdf_temp["overlap__seq"].str.len()/pdf_temp["seq__dna_x"].str.len()
    print("    * Overlaps calculated!")
    for i in pdf_temp.sort_values(["Locus", "overlap__perc"], ascending=[True, False]).groupby("Locus").head(1)[["Locus","overlap__perc"]].values:
        print(f"      - Locus: {i[0]} | Overlap: {i[1]}")


In [18]:
sequence = dict_sequence["NC_045512.2"]

In [29]:
main(sequence)


 -> Sequence len: 29903

 -> Frequence:
    A: 29.94%
    T: 32.08%
    G: 19.61%
    C: 18.37%

 -> GC content: 37.97%

 -> Number of start/end codons:
    * In frame 0: 
      - Start: 117 | Stop: 176
    * In frame 1: 
      - Start: 311 | Stop: 136
    * In frame 2: 
      - Start: 297 | Stop: 172

 -> Most and less freq. codon:
    TTT: 1004
    CCG: 74

 -> creating all_potential_proteins.txt file...
    * file created!

 -> creating orf_coordinates.txt file...
    * file created!

 -> Getting overlap between ORFs...
    * Overlaps calculated!
      - Locus: E | Overlap: 1.0
      - Locus: M | Overlap: 1.0
      - Locus: N | Overlap: 1.0
      - Locus: ORF10 | Overlap: 0.06837606837606838
      - Locus: ORF1ab | Overlap: 1.0
      - Locus: ORF3a | Overlap: 1.0
      - Locus: ORF6 | Overlap: 1.0
      - Locus: ORF7a | Overlap: 1.0
      - Locus: ORF7b | Overlap: 1.0
      - Locus: ORF8 | Overlap: 1.0
      - Locus: S | Overlap: 1.0
